In [1]:
import pandas as pd
import numpy as np

In [2]:
FILE = 'travistorrent_11_1_2017.csv'
#size = 500000
size = 3000000

In [3]:
#for dataset in pd.read_csv(file, engine='c', chunksize=size):
#    process(dataset)

In [4]:
%%time
dataset = pd.read_csv(FILE, sep=',', engine='c',dtype={'gh_team_size': np.int32,
                                                        'git_num_all_built_commits': np.int32,
                                                        'gh_num_commit_comments': np.int32,
                                                        'git_diff_src_churn': np.int32,
                                                        'git_diff_test_churn': np.int32,
                                                        'gh_diff_files_added': np.int32,
                                                        'gh_diff_files_deleted': np.int32,
                                                        'gh_diff_files_modified': np.int32,
                                                        'gh_diff_tests_added': np.int32,
                                                        'gh_diff_tests_deleted': np.int32,
                                                        'gh_diff_src_files': np.int32,
                                                        'gh_diff_doc_files': np.int32,
                                                        'gh_diff_other_files': np.int32,
                                                        'gh_num_commits_on_files_touched': np.int32,
                                                        'gh_sloc': np.int32,
                                                        'tr_build_number': np.int32,
                                                        'tr_job_id': np.int32,
                                                        'tr_build_id': np.float32,
                                                        'gh_pull_req_num': np.float32,
                                                        'gh_num_commits_in_push': np.float32,
                                                        'tr_prev_build': np.float32,
                                                        'gh_num_issue_comments': np.float32,
                                                        'gh_num_pr_comments': np.float32,
                                                        'gh_test_lines_per_kloc': np.float32,
                                                        'gh_test_cases_per_kloc': np.float32,
                                                        'gh_asserts_cases_per_kloc': np.float32,
                                                        'gh_description_complexity': np.float32,
                                                        'tr_duration': np.float32,
                                                        'tr_log_setup_time': np.float32,
                                                        'tr_log_num_tests_ok': np.float32,
                                                        'tr_log_num_tests_failed': np.float32,
                                                        'tr_log_num_tests_run': np.float32,
                                                        'tr_log_num_tests_skipped': np.float32,
                                                        'tr_log_testduration': np.float32,
                                                        'tr_log_buildduration': np.float32,

                                                        'gh_project_name': 'category',
                                                        'gh_pr_created_at': 'category',
                                                        'gh_lang': 'category',
                                                        'git_merged_with': 'category',
                                                        'git_branch': 'category',
                                                        'gh_commits_in_push': 'category',
                                                        'git_prev_commit_resolution_status': 'category',
                                                        'git_prev_built_commit': 'category',
                                                        'gh_first_commit_created_at': 'category',
                                                        'git_all_built_commits': 'category',
                                                        'git_trigger_commit': 'category',
                                                        'tr_virtual_merged_into': 'category',
                                                        'tr_original_commit': 'category',
                                                        'gh_pushed_at': 'category',
                                                        'gh_build_started_at': 'category',
                                                        'tr_status': 'category',
                                                        'tr_jobs': 'category',
                                                        'tr_log_lan': 'category',
                                                        'tr_log_status': 'category',
                                                        'tr_log_analyzer': 'category',
                                                        'tr_log_frameworks': 'category',
                                                        'tr_log_bool_tests_failed': 'category',
                                                        'tr_log_tests_failed': 'category',


                                                        'gh_is_pr': np.bool,
                                                        'gh_by_core_team_member': np.bool,
                                                        'tr_log_bool_tests_ran': np.bool}

                                                                                          , nrows = size, encoding='utf-8')

'''
Signature:
    sep - separa por virgula
    nrows - limite de linhas lidas
    encoding - tipo de caracter
    engine - Mecanismo do analisador
    na_values - procura os padrões na|nan como str
    keep_default_na - define como nan os padroes econtrados em na_values
    
'''



CPU times: user 58.1 s, sys: 2.1 s, total: 1min
Wall time: 1min 3s


In [5]:
# props, NAlist = reduce_mem_usage(pd.read_csv(FILE))

<font size=4/>
   * Elimia as colunas não utilizadas

In [6]:
# Eliminando colunas - 
dataset.drop([  'gh_diff_files_added',
                'gh_commits_in_push',
                'gh_diff_files_deleted',
                'gh_diff_files_modified',
                'gh_diff_tests_added',
                'gh_diff_tests_deleted',
                'gh_diff_src_files',
                'gh_diff_doc_files',
                'gh_diff_other_files',
                'gh_sloc',
                'gh_test_lines_per_kloc',
                'gh_test_cases_per_kloc',
                'gh_pull_req_num',
                
                'tr_status',
                'tr_job_id',
                'tr_jobs',
                'tr_build_number',
                'tr_duration',
                'tr_log_lan',
                'tr_log_num_tests_run',
                'tr_log_setup_time',
                'tr_log_analyzer',
                'tr_log_frameworks',
                'tr_log_bool_tests_ran',
                'tr_log_bool_tests_failed',
                'tr_log_tests_failed',
                'tr_log_testduration',
                'tr_log_buildduration'], axis=1, inplace=True)

In [9]:
#df_ruby = dataset.loc[dataset['gh_project_name']=='rails/rails']

df_ruby = dataset.dropna() # Remove as linhas NAN

df_ruby['date'], df_ruby['hours'] = df_ruby.gh_pushed_at.str.split(' ').str # Separa a coluna que possui data e hora em duas novas colunas de data hora
    

<font size=4/> 
   * Filtra o projeto, remove linhas NAN, separa a coluna gh_pushed_at

In [12]:
projects = dataset.drop_duplicates(['gh_project_name'])
projects = projects.loc[projects['gh_lang']=='ruby']
projects = projects.gh_project_name


In [13]:
def taxa_commit(project_name):
    
    df_ruby = dataset.loc[dataset['gh_project_name'] == project_name] # Filtro penas o projeto
    # remove linhas duplicadas
    
    #single_push = df_ruby.drop_duplicates(['tr_build_id']) # remove as linhas duplicadas
    #single_push = single_push['tr_build_id'] # cria uma serie com os push unicos do projeto
    start_build = df_ruby.tr_build_id
    
    # soma todos pushs
    num_commit = df_ruby.gh_num_commits_in_push.sum() 
    
    # Duração do projeto
    first_data, hora = df_ruby.gh_first_commit_created_at.head(1).str.split(' ').str
    first_data = int(first_data.str.replace('-', ''))
    
    last_data,  hora = df_ruby.gh_first_commit_created_at.tail(1).str.split(' ').str
    last_data = int(last_data.str.replace('-',''))
    
    date_commit = last_data - first_data
    
    # Media participante
    
    mean_participante = df_ruby.gh_team_size.sum()/ len(start_build)
    
    # Taxa de commit
    
    taxa_commit = num_commit/(date_commit * mean_participante)
    
    return(taxa_commit)

In [14]:
lista_taxa_commit = []
for project_name in projects:
    #print(project_name)
     lista_taxa_commit.append(taxa_commit(project_name))

/home/wagner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in true_divide


[0.003788323929617337, 0.002274038836963666, 0.0034935515559517214, 0.006710691729457383, 0.006998023296239102, 0.00896136462953657, 0.002245816788650118, 0.008398172520419056, 0.0021680479258871367, 0.015388034449720589, 0.0043246612258372105, 0.017685166153351542, 0.0020810253068317586, 0.007095637780389474, 0.006134852807712228, 0.025554410817388823, 0.0003126154656270426, 0.010876152631023449, 0.003029346952829286, 0.005355034061683597, 0.0005221326216859083, 0.004002826355767532, 0.1230405205560485, 0.0007693930281030503, 0.0008191854523457084, 0.0007908227547788827, 0.0033989120096546725, 0.0014106146043192534, 0.002777312962390535, 0.0013813988511662684, 0.0032894766663372804, 0.00826993268206105, 0.0034225634142735616, 0.0007704675122480289, 0.013467655045520544, 0.003165881329934375, 0.004050890585241731, 0.004520872152908057, 0.010778522024901552, 0.0016024251222349658, 0.0056276051408089, 0.005480384357530778, 0.008325028823203467, 0.0013119404827220255, 0.000182603932071337

In [58]:
one = taxa_commit(project_name)

one

0.014350276795791908

<font size=4/> 
   * Remove linhas duplicadas

In [25]:
#single_push = df_ruby.drop_duplicates(['tr_build_id']) # remove as linhas duplicadas

#single_push = single_push['tr_build_id'] # cria uma serie com os push unicos do projeto

start_build = df.ruby.tr_build_id

<font size=4/> 
   * Soma de todos push

In [26]:
num_commit = df_ruby.gh_num_commits_in_push.sum() 

<font size=4/> 
   * Duração do projeto

In [27]:
first_data, hora = df_ruby.gh_first_commit_created_at.head(1).str.split(' ').str
first_data = int(first_data.str.replace('-', ''))

In [28]:
last_data,  hora = df_ruby.gh_first_commit_created_at.tail(1).str.split(' ').str
last_data = int(last_data.str.replace('-',''))

In [33]:
date_commit = last_data - first_data

<font size=4/> 
   * Media participante

In [34]:
mean_participante = df_ruby.gh_team_size.sum()/ len(start_build)

<font size=4/> 
   * Taxa de commit

In [35]:
taxa_commit = num_commit/(date_commit * mean_participante)
taxa_commit

0.014350276795791908

In [37]:
df_ruby.head(5)

,tr_build_id,gh_project_name,gh_is_pr,gh_pr_created_at,gh_pull_req_num,gh_lang,git_merged_with,git_branch,gh_num_commits_in_push,git_prev_commit_resolution_status,...,gh_pushed_at,gh_build_started_at,tr_status,tr_job_id,tr_log_status,tr_log_num_tests_ok,tr_log_num_tests_failed,tr_log_num_tests_skipped,data,hora
638680,8113937.0,rails/rails,True,2013-06-12 10:21:13,10925.0,ruby,merge_button,master,2.0,build_found,...,2013-02-17 01:06:32,2013-06-15 12:47:57,passed,8113939,ok,14749.0,0.0,2.0,2013-02-17,01:06:32
638681,8113937.0,rails/rails,True,2013-06-12 10:21:13,10925.0,ruby,merge_button,master,2.0,build_found,...,2013-02-17 01:06:32,2013-06-15 12:47:57,passed,8113941,ok,7260.0,0.0,44.0,2013-02-17,01:06:32
638682,8113937.0,rails/rails,True,2013-06-12 10:21:13,10925.0,ruby,merge_button,master,2.0,build_found,...,2013-02-17 01:06:32,2013-06-15 12:47:57,passed,8113943,ok,7630.0,0.0,77.0,2013-02-17,01:06:32
638683,8113937.0,rails/rails,True,2013-06-12 10:21:13,10925.0,ruby,merge_button,master,2.0,build_found,...,2013-06-15 12:45:17,2013-06-15 12:47:57,passed,8113944,ok,879.0,0.0,0.0,2013-06-15,12:45:17
638684,8113937.0,rails/rails,True,2013-06-12 10:21:13,10925.0,ruby,merge_button,master,2.0,build_found,...,2013-06-15 12:45:17,2013-06-15 12:47:57,passed,8113949,ok,7464.0,0.0,96.0,2013-06-15,12:45:17
